In [12]:
import datasets
from langchain.docstore.document import Document
from langchain_community.retrievers import BM25Retriever
from langchain.tools import Tool
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from llama_index.llms.openai import OpenAI
from langchain_openai import ChatOpenAI


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

open_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

In [5]:
bm25_retriever = BM25Retriever.from_documents(docs)

In [6]:
def extract_text(query: str) -> str:
    """Retrieves detailed information about gala guests based on their name or relation."""
    results = bm25_retriever.invoke(query)
    if results:
        return "\n\n".join([doc.page_content for doc in results[:3]])
    else:
        return "No matching guest information found."

guest_info_tool = Tool(
    name="guest_info_retriever",
    func=extract_text,
    description="Retrieves detailed information about gala guests based on their name or relation."
)

In [17]:
# Initialize the model with the required parameters
openai_model_name =  "gpt-4o-mini"
model = OpenAI(model=openai_model_name, api_key=open_api_key)

chat = ChatOpenAI(model=openai_model_name, api_key=open_api_key, temperature=0)


In [18]:
tools = [guest_info_tool]
chat_with_tools = chat.bind_tools(tools)

In [19]:
# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [20]:
def assistant(state: AgentState):
    return {
        "messages": [chat_with_tools.invoke(state["messages"])],
    }

In [21]:
## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Tell me about our guest named 'Lady Ada Lovelace'.")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Lady Ada Lovelace is your best friend and an esteemed mathematician. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her contributions to Charles Babbage's Analytical Engine. 

Here are her details:
- **Name:** Ada Lovelace
- **Relation:** Best friend
- **Description:** Esteemed mathematician and friend, known for her pioneering work in mathematics and computing.
- **Email:** ada.lovelace@example.com
